In [1]:
import numpy as np
import pandas as pd

In [2]:
finname_fimo = 'matrix_classic_FIMO_1e-5_masterlist_DHSs_733samples_WM20180608_all_overlapping.hg38.txt'

it is probably possible to do this in batches... may be necessary if this fails 

In [3]:
FIMO = pd.read_table(finname_fimo)

/home/amuratov/.local/lib/python3.5/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.


In [4]:
FIMO.shape

(3591898, 2089)

In [5]:
FIMOcolnames = FIMO.columns.values

In [7]:
len(FIMOcolnames)

2089

In [8]:
FIMOcolnames[0:20]

array(['#chrom', 'min0', 'max1', 'TOTAL', 'Alx1.mouse_homeodomain_1',
       'Alx1.mouse_homeodomain_2', 'Alx3_3418.2', 'ALX3_homeodomain_1',
       'ALX3_homeodomain_2', 'ALX3_homeodomain_3', 'Alx4_1744.1',
       'ALX4_homeodomain_1', 'Alx4.mouse_homeodomain_1', 'Arid3a_primary',
       'Arid3a_secondary', 'Arid5a_primary', 'Arid5a_secondary',
       'Ar.mouse_nuclearreceptor_1', 'ARNTL_bHLH_1',
       'AR_nuclearreceptor_1'], dtype=object)

took about 20 min to read

#### 2089 coloumns in the FIMO scan matrix, but the first 4 are not motif clusters. So 2085 total notifs in the FIMO matrix

In [9]:
cluster_lookup_name = 'clusters.long.txt'
clustDF = pd.read_table(cluster_lookup_name, names=['clust_name', 'motif_name', 'sign', 'number'])
clustnames_ar = clustDF.clust_name.value_counts().index.values


/home/amuratov/.local/lib/python3.5/site-packages/ipykernel_launcher.py:2: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  


In [10]:
len(clustnames_ar)

283

In [11]:
clustDF.shape

(2316, 4)

#### 2316 motifs listed in the cluster names file

In [13]:
FIMOcolnames.astype(str)

array(['#chrom', 'min0', 'max1', ..., 'ZSCAN4_C2H2_1', 'Zscan4_primary',
       'Zscan4_secondary'], dtype='<U29')

## In order to match things up, we need to capitalize the names of the FIMO columns

In [29]:
def uppercaser(list_of_strings):
    newlist = []
    for item in list_of_strings:
        newlist.append(item.upper())
    return np.array(newlist)


In [33]:
FIMO_UP_colnames_ar = uppercaser(FIMOcolnames.astype(str).tolist())

In [70]:
FIMO_UP_colnames_ar[0:20]

array(['#CHROM', 'MIN0', 'MAX1', 'TOTAL', 'ALX1.MOUSE_HOMEODOMAIN_1',
       'ALX1.MOUSE_HOMEODOMAIN_2', 'ALX3_3418.2', 'ALX3_HOMEODOMAIN_1',
       'ALX3_HOMEODOMAIN_2', 'ALX3_HOMEODOMAIN_3', 'ALX4_1744.1',
       'ALX4_HOMEODOMAIN_1', 'ALX4.MOUSE_HOMEODOMAIN_1', 'ARID3A_PRIMARY',
       'ARID3A_SECONDARY', 'ARID5A_PRIMARY', 'ARID5A_SECONDARY',
       'AR.MOUSE_NUCLEARRECEPTOR_1', 'ARNTL_BHLH_1',
       'AR_NUCLEARRECEPTOR_1'], dtype='<U29')

In [67]:
cluster_table = []
total_match_list = []
for cluster in clustnames_ar:
    print('doing ',cluster)
    relevantcols_cut = clustDF.clust_name.values == cluster
    relevant_cols = clustDF.motif_name.values[relevantcols_cut]
    relevant_cols_match = np.intersect1d(relevant_cols, FIMO_UP_colnames_ar)
    which_FIMOs_are_in_this_cluster = np.in1d(FIMO_UP_colnames_ar, relevant_cols_match)
    total_match_list += FIMO_UP_colnames_ar[which_FIMOs_are_in_this_cluster].tolist()
    cluster_table.append(FIMO[FIMO.columns[which_FIMOs_are_in_this_cluster]].sum(axis=1).values)

doing  c278-HD/5
doing  c25-ETS
doing  c75-Forkhead
doing  c265-HD/2
doing  c90-MYC/MAX
doing  c269-HD/3
doing  c64.ESRRA/2
doing  c39-SOX
doing  c85-NR/2
doing  c68-CREB/ATF
doing  c112-Ebox
doing  c91-T-box
doing  c184-E2F/2
doing  c49-CCAAT/CEBP
doing  c113-MEIS
doing  c136-EGR/KLF/Sp1
doing  c36-NKX
doing  c203-AP-2/2
doing  c84-NR
doing  c66-AP-1
doing  c18-POU
doing  c57-SOX/3
doing  c283-HD/6
doing  c29-STAT
doing  c82-GR
doing  c116-GLI
doing  c134-NF-kB
doing  c93-MAF
doing  c80-GATA/2
doing  c274-HNF1/POU
doing  c56-SOX/2
doing  c107-POU/2
doing  c188-RUNX/AML
doing  c145-IRF
doing  c117-GLI/2
doing  c137-Sp1
doing  c272-SOX/5
doing  c42.CUX1
doing  c121-SIX
doing  c62.ESRRA
doing  c70.MEF2A/2
doing  c101.IRX2
doing  c59-GATA
doing  c151-ZNF
doing  c28-NFAT
doing  c222-CTF/NF-I
doing  c55.SRF
doing  c31-TCF/LEF
doing  c65-NEURO/OLIG
doing  c40.YY1
doing  c109-Ebox/CACCTG
doing  c162.TP53/2
doing  c207-RFX/3
doing  c275.LEF1
doing  c205-RFX
doing  c198.HSF1
doing  c157-IRF/2
d

### verify that all the FIMO column names matched, and they all matched uniquely

In [68]:
len(total_match_list)

2085

In [69]:
len(list(set(total_match_list)))

2085

# Now to save our results

In [9]:
cluster_table = np.array(cluster_table)

In [15]:
cluster_tableDF = pd.DataFrame(cluster_table.T, columns=clustnames_ar, dtype=int)

In [16]:
cluster_tableDF.shape

(3591898, 283)

In [17]:
cluster_tableDF.head()

,c278-HD/5,c25-ETS,c75-Forkhead,c265-HD/2,c90-MYC/MAX,c269-HD/3,c64.ESRRA/2,c39-SOX,c85-NR/2,c68-CREB/ATF,...,c167.PAX4/3,c161.SMAD3,c225.PUR1,c105.FOXP1,c245.AP3,c263.DEAF1/2,c152.AIRE/2,c260.ZFP161,c1.P53,c210.PAX3
0,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,14,0,0,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
cluster_tableDF.to_csv('20619_FIMOtable_clusters.txt', sep='\t', index=False)

for some stupid reason, the numpy is way slower

In [19]:
np.save('20619_cluster_table.npy', cluster_tableDF.values.astype(int))